In [14]:
!conda install -c conda-forge geopy --yes 

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.19.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.12 MB/s
geopy-1.19.0-p 100% |################################| Time: 0:00:00  35.83 MB/s


In [15]:
# import all needed libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [4]:
# Grab wikipedia page
wikiraw=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [5]:
# Parse html page using 'lxml parser'
wikiparsed = BeautifulSoup(wikiraw,'lxml')

In [6]:
# Find table that nees to be converted into dataframe
ca_postal_tbl = wikiparsed.find('table', class_='wikitable sortable').tbody 

In [7]:
# create empty lists and append all parsed row elements
n=0
cols=[]
PostalCode=[]
Borough=[]
Neighborhood=[]
rcount=len(ca_postal_tbl.findChildren(recursive=False))
while n < rcount:
      trow = ca_postal_tbl.findChildren(recursive=False)[n]
      if n!=0: 
        PostalCode.append(trow.find_all('td')[0].text.replace('\n','').replace("'",""))
        Borough.append(trow.find_all('td')[1].text.replace('\n',''))
        Neighborhood.append(trow.find_all('td')[2].text.replace('\n',''))  
      n=n+1

In [8]:
# create dataframe and add lists as columns
df=pd.DataFrame()
df['PostalCode']=PostalCode
df['Borough']=Borough
df['Neighborhood']=Neighborhood

In [9]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [10]:
# Filter out dataframe with Borough column of value 'Not assigned'
df=df[(df.Borough!='Not assigned')]

In [11]:
# Group by columns 'PostalCode','Borough' and concatenate neighborhood values
df2=df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()

In [12]:
# dataframe print few rows
df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
# CA neighborhood dataframe shape
df2.shape

(103, 3)

In [34]:
# Create dataframe br reading from geospatial CSV file
GeoData = pd.read_csv('http://cocl.us/Geospatial_data')

In [35]:
# Read few rows from geodata dataframe
cols=['PostalCode','Latitude','Longitude']
GeoData.columns=cols
GeoData.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
# print shape pf dataframe
GeoData.shape

(103, 3)

In [37]:
# Merge both dataframes df2 and GeoData based on common column PostalCode
df3=pd.merge(df2,GeoData,how='left',on='PostalCode')
df3.shape

(103, 5)

In [38]:
# Print few rows from newly merged dataframe. New Dataframe df3 contains Latitude and Longitude info
df3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
